Cancer Site Analysis
===

In [1]:
%matplotlib inline
from IPython.core.display import display, HTML

import os
import numpy as np
import pandas as pd
import itertools

import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib
import pylab as pl

import datetime as dt
from datetime import datetime
import pytz
import time

from collections import Counter

import json
import os
import re
from html.parser import HTMLParser
import itertools
import multiprocessing as mp
from nltk import word_tokenize
from IPython.core.display import display, HTML
import datetime as dt

In [2]:
pd.read_feather

<function pandas.io.feather_format.read_feather(path, nthreads=1)>

In [3]:
working_dir = "/home/srivbane/shared/caringbridge/data/projects/qual-health-journeys/extract_site_features"
feathered_site_df_filename = os.path.join(working_dir, "site_scrubbed.df")
df = pd.read_feather(feathered_site_df_filename)

In [4]:
min(df["createdAt"])

1117605558000.0

In [5]:
len(df[df["healthCondition_category"] == "Cancer"])

104500

In [6]:
cancer_df = df[df["healthCondition_category"] == "Cancer"]

In [7]:
np.sum(cancer_df["numJournals"])

4389786.0

In [8]:
# load the actual counts
filename = "/home/srivbane/shared/caringbridge/data/raw/../projects/classify_health_condition/vw_experiments/site_survival_time.csv"
count_df = pd.read_csv(filename, header=None, names=["_id", "created_at_utc", "updated_at_utc", "first_journal_timestamp", "last_journal_timestamp", "numJournals", "trueNumJournals"])

In [9]:
count_df.head()

,_id,created_at_utc,updated_at_utc,first_journal_timestamp,last_journal_timestamp,numJournals,trueNumJournals
0,1,1.117606e+09,1.371331e+09,1.103857e+09,1.261008e+09,180,187
1,3,1.117606e+09,1.371066e+09,1.117606e+09,1.117606e+09,1,1
2,4,1.117606e+09,1.370884e+09,1.117606e+09,1.147749e+09,2,2
3,2,1.118088e+09,1.371138e+09,1.117053e+09,1.180150e+09,0,29
4,8,1.117608e+09,1.371260e+09,1.117608e+09,1.117608e+09,1,1


In [10]:
merged = pd.merge(cancer_df, count_df, how='left', on="_id")

In [11]:
merged[["numJournals_x", "numJournals_y", "trueNumJournals"]].describe()

,numJournals_x,numJournals_y,trueNumJournals
count,101775.000000,104500.000000,104500.000000
mean,43.132262,41.981445,47.114890
std,75.918657,75.251640,104.611362
min,0.000000,-1.000000,0.000000
25%,7.000000,6.000000,7.000000
50%,22.000000,21.000000,21.000000
75%,49.000000,48.000000,50.000000
max,3762.000000,3762.000000,5034.000000


In [12]:
np.sum(merged["trueNumJournals"])

4923506

In [13]:
utc1990 = 631152000
utc2000 = 946684800

In [14]:
of_interest = merged[(merged["trueNumJournals"] >= 3) & (merged["last_journal_timestamp"] >= utc1990) & (merged["first_journal_timestamp"] >= utc1990)]
len(of_interest)

87234

In [15]:
# This should be the duration in years of the longest site, but it seems way too long...
np.max(of_interest["last_journal_timestamp"] - of_interest["first_journal_timestamp"]) / 60 / 60 / 24 / 365

19.00298706240487

In [16]:
of_interest["survival_time"] = of_interest["last_journal_timestamp"] - of_interest["first_journal_timestamp"]
longest_cancer_sites = of_interest[["_id", "created_at_utc", "updated_at_utc", "first_journal_timestamp", "last_journal_timestamp", "survival_time"]]
longest_cancer_sites = longest_cancer_sites.sort_values(by="survival_time", ascending=False)
longest_cancer_sites.head()

/home/srivbane/levon003/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,_id,created_at_utc,updated_at_utc,first_journal_timestamp,last_journal_timestamp,survival_time
82092,644167,1.356228e+09,1.371276e+09,1.357269e+09,1.956547e+09,599278200.0
3300,17812,1.153181e+09,1.458358e+09,9.467597e+08,1.461687e+09,514927049.0
10076,32515,1.175393e+09,1.371394e+09,9.557909e+08,1.461887e+09,506096571.0
72148,558493,1.341144e+09,1.371381e+09,9.292757e+08,1.424269e+09,494992854.0
15304,43301,1.186368e+09,1.370898e+09,9.602203e+08,1.449791e+09,489570526.0


In [17]:
def convert_dates(created_at_utc):
    if created_at_utc < utc1990:
        return "%.2f years" % (created_at_utc / (60 * 60 * 24 * 365))
    
    unaware_datetime = datetime.utcfromtimestamp(created_at_utc)
    utc_datetime = unaware_datetime.replace(tzinfo=pytz.UTC)
    cst_datetime = utc_datetime.astimezone(tz=pytz.timezone("America/Chicago"))
    
    datetime_string = cst_datetime.strftime("%Y-%m-%d")
    return datetime_string

str_longest_df = longest_cancer_sites.copy()
for col in ['created_at_utc', 'updated_at_utc', 'first_journal_timestamp', 'last_journal_timestamp', 'survival_time']:
    str_longest_df[col] = str_longest_df[col].map(convert_dates)

In [18]:
str_longest_df.head(n=30)

,_id,created_at_utc,updated_at_utc,first_journal_timestamp,last_journal_timestamp,survival_time
82092,644167,2012-12-22,2013-06-15,2013-01-03,2031-12-31,19.00 years
3300,17812,2006-07-17,2016-03-18,2000-01-01,2016-04-26,16.33 years
10076,32515,2007-03-31,2013-06-16,2000-04-15,2016-04-28,16.05 years
72148,558493,2012-07-01,2013-06-16,1999-06-13,2015-02-18,15.70 years
15304,43301,2007-08-05,2013-06-10,2000-06-05,2015-12-10,15.52 years
3213,17605,2006-07-13,2013-05-27,2000-12-02,2016-02-08,15.19 years
10656,33704,2007-04-12,2013-04-07,2001-12-07,2016-03-29,14.32 years
52167,381184,2011-08-18,2013-05-22,2011-08-21,2025-11-30,14.29 years
3945,19308,2006-08-15,2013-06-15,2001-01-01,2014-12-31,14.01 years
1663,11157,2006-03-01,2013-06-12,2001-01-27,2014-09-10,13.63 years


In [19]:
len(longest_cancer_sites[longest_cancer_sites["created_at_utc"] > longest_cancer_sites["first_journal_timestamp"]])

7923

In [20]:
# median survival time
convert_dates(np.median(of_interest["survival_time"]))

'0.64 years'